# Working with Raster Bands

## Preparing Your Workspace

### Option 1: (recommended) Run in Google Colab
[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/kevinlacaille/presentations/blob/main/scipy2024/6_image_processing.ipynb)

### Option 2: Run local Jupyter instance
You can also choose to open this Notebook in your own local Jupyter instance.

**Prerequisites**

- Install: rasterio, exiftool
- Download data

In [ ]:
!pip install rasterio
!apt-get install -y exiftool
!pip install PyExifTool
!wget https://raw.githubusercontent.com/kevinlacaille/presentations/main/scipy2024/data/presentation/8928dec4ddbffff/DJI_0876.JPG
# Download the zip file
!wget https://github.com/kevinlacaille/presentations/raw/main/scipy2024/data/presentation/vancouver_data.zip -O /content/vancouver_data.zip
# Unzip the file
!unzip /content/vancouver_data.zip -d /content/vancouver_data

In [ ]:
import rasterio
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv
import exiftool
import json

In [ ]:
np.seterr(divide='ignore', invalid='ignore')

## Define functions
Define functions for each process used in notebooks 1-5.

In [ ]:
def get_bands(image_path):
    """
    Open a geospatial image and read the blue, green, and red bands.

    Parameters:
    -----------
    image_path : str
        The file path to the geospatial image.

    Returns:
    --------
    blue : numpy.ndarray
        The blue band of the image as a NumPy array.
    green : numpy.ndarray
        The green band of the image as a NumPy array.
    red : numpy.ndarray
        The red band of the image as a NumPy array.
    rgb : numpy.ndarray
        A 3-dimensional array combining the blue, green, and red bands into an RGB image.
    """

    # Open the image and read the bands as numpy arrays
    with rasterio.open(image_path) as src:
        blue = src.read(1)
        green = src.read(2)
        red = src.read(3)

    rgb = np.dstack((blue, green, red))

    return blue, green, red, rgb

In [ ]:
def get_metadata(image_path):
    """
    Extract metadata from a geospatial image.

    Parameters:
    -----------
    image_path : str
        The file path to the geospatial image.

    Returns:
    --------
    metadata : dict
        A dictionary containing the metadata of the image.
    """

    # Get the metadata of the image
    with exiftool.ExifTool() as et:
        metadata = json.loads(et.execute(b'-j', image_path))

    return metadata

In [ ]:
def get_gsd(metadata, height=15):
    """
    Calculate the Ground Sampling Distance (GSD) for a given image,
    both at the altitude and at a specified height above ground.

    Parameters:
    -----------
    metadata : dict
        A dictionary containing the metadata of the image.
    height : float, optional
        The height above ground for which to calculate the GSD, in meters. Default is 15 meters.

    Returns:
    --------
    gsd : float
        The GSD at the altitude of the image capture, in meters per pixel.
    gsd_tree : float
        The GSD at the specified height above ground, in meters per pixel.
    """

    # Extract the GPS Altitude
    altitude = float(metadata[0].get("XMP:RelativeAltitude"))
    print(f"Altitude: {altitude} m")

    # Extract the focal length of the camera
    focal_length = metadata[0].get("EXIF:FocalLength")  # in mm
    # Extract the image's width
    image_width = metadata[0].get("File:ImageWidth")

    # Compute the pixel pitch
    pixel_pitch = 6.17e-3 / image_width

    # Compute the global GSD and the GSD at the tree top
    gsd = (altitude) * pixel_pitch / (focal_length / 1000)
    gsd_tree = (altitude - height) * pixel_pitch / (focal_length / 1000)

    return gsd, gsd_tree

In [ ]:
def get_kernel_size(gsd_tree):
    """
    Calculate the kernel size for morphological operations based on the GSD at tree height.

    Parameters:
    -----------
    gsd_tree : float
        The Ground Sampling Distance at the height of the tree, in meters per pixel.

    Returns:
    --------
    area_of_tree : float
        The area of the tree in pixels.
    kernel_size : int
        The kernel size for morphological operations, adjusted to be an odd number.
    """

    # Diameter of a tree
    diameter_of_tree = 10  # meters

    # Number of pixels the tree will cover
    diameter_of_tree_px = diameter_of_tree / gsd_tree
    area_of_tree = np.pi * (diameter_of_tree_px / 2)**2

    # Kernel size for morphological operations
    kernel_size = int(np.sqrt(diameter_of_tree_px))
    # Ensure kernel size is odd
    if kernel_size % 2 == 0:
        kernel_size += 1
    print(f"Kernel size: {kernel_size}")

    return area_of_tree, kernel_size

In [ ]:
def get_vari(blue, green, red):
    """
    Calculate the Visible Atmospherically Resistant Index (VARI)
    using the blue, green, and red bands of an image.

    Parameters:
    -----------
    blue : numpy.ndarray
        The blue band of the image as a NumPy array.
    green : numpy.ndarray
        The green band of the image as a NumPy array.
    red : numpy.ndarray
        The red band of the image as a NumPy array.

    Returns:
    --------
    vari : numpy.ndarray
        The calculated VARI index as a NumPy array.
    """

    # Calculate the VARI index
    vari = (green.astype(float) - red.astype(float)) / (
        green.astype(float) + red.astype(float) - blue.astype(float))

    return vari

In [ ]:
def threshold(vari, vari_min=0.1):
    """
    Generate vegetation and non-vegetation masks based on VARI index thresholds.

    Parameters:
    -----------
    vari : numpy.ndarray
        The VARI index as a NumPy array.
    vari_min : float, optional
        The minimum threshold for the VARI index to classify vegetation. Default is 0.1.

    Returns:
    --------
    vegetation_mask : numpy.ndarray
        A mask where vegetation areas are marked with 1 and non-vegetation areas are NaN.
    non_vegetation_mask : numpy.ndarray
        A mask where non-vegetation areas are marked with 1 and vegetation areas are NaN.
    """

    # Generate the vegetation mask
    vegetation_mask = np.full(vari.shape, np.nan)
    vegetation_mask[(vari >= vari_min)] = 1

    # Generate the non-vegetation mask
    non_vegetation_mask = np.full(vari.shape, np.nan)
    non_vegetation_mask[vari < vari_min] = 1

    return vegetation_mask, non_vegetation_mask

In [ ]:
def smoothing(mask, kernel_size=7):
    """
    Apply Gaussian blur to smooth the edges of a mask.

    Parameters:
    -----------
    mask : numpy.ndarray
        The binary mask to be smoothed.
    kernel_size : int, optional
        The size of the kernel to be used in the Gaussian blur. Default is 7.

    Returns:
    --------
    blur : numpy.ndarray
        The smoothed mask after applying Gaussian blur.
    """

    # Apply a Gaussian blur to the mask
    blur = cv.GaussianBlur(mask, (kernel_size, kernel_size), 0)

    return blur

In [ ]:
def morphological_operations(mask, kernel_size=19):
    """
    Apply opening and closing morphological operations to a mask.

    Parameters:
    -----------
    mask : numpy.ndarray
        The binary mask to be processed.
    kernel_size : int, optional
        The size of the kernel to be used in the morphological operations. Default is 19.

    Returns:
    --------
    closing : numpy.ndarray
        The mask after applying opening followed by closing morphological operations.
    """

    # Define the kernel size
    opening_kernel = np.ones((kernel_size, kernel_size), np.uint8)
    closing_kernel = np.ones((kernel_size, kernel_size), np.uint8)

    # Apply the morphological filters
    opening = cv.morphologyEx(mask, cv.MORPH_OPEN, opening_kernel)
    closing = cv.morphologyEx(opening, cv.MORPH_CLOSE, closing_kernel)

    return closing

In [ ]:
def segmentation(mask, rgb):
    """
    Apply a mask to the RGB image to create a segmentation overlay and its inverse.

    Parameters:
    -----------
    mask : numpy.ndarray
        The binary mask to be applied to the RGB image.
    rgb : numpy.ndarray
        The original RGB image.

    Returns:
    --------
    mask_overlay : numpy.ndarray
        The RGB image with the mask overlay applied, highlighting the masked areas in purple.
    inverse_mask_overlay : numpy.ndarray
        The RGB image with the inverse mask overlay applied, highlighting the non-masked areas in purple.
    """

    # Apply the mask to the RGB image
    mask_overlay = np.zeros_like(rgb)
    mask_overlay[:, :, 0] = 128  # Red channel for purple
    mask_overlay[:, :, 1] = 0  # Green channel for purple
    mask_overlay[:, :, 2] = 128  # Blue channel for purple

    # Apply the filtered mask to the mask overlay
    mask_overlay[mask != 1] = [0, 0, 0]

    # Create the inverse mask
    inverse_mask = np.logical_not(mask).astype(np.uint8)

    # Create an inverse mask overlay with purple color
    inverse_mask_overlay = np.zeros_like(rgb)
    inverse_mask_overlay[:, :, 0] = 128  # Red channel for purple
    inverse_mask_overlay[:, :, 1] = 0  # Green channel for purple
    inverse_mask_overlay[:, :, 2] = 128  # Blue channel for purple

    # Apply the inverse mask to the inverse mask overlay
    inverse_mask_overlay[mask == 1] = [0, 0, 0]

    return mask_overlay, inverse_mask_overlay

In [ ]:
def visualize_segmentation(rgb, mask_overlay, inverse_mask_overlay):
    """
    Visualize the original RGB image, vegetation segmentation, and non-vegetation segmentation.

    Parameters:
    -----------
    rgb : numpy.ndarray
        The original RGB image.
    mask_overlay : numpy.ndarray
        The RGB image with the vegetation mask overlay applied.
    inverse_mask_overlay : numpy.ndarray
        The RGB image with the non-vegetation mask overlay applied.

    Returns:
    --------
    None
    """

    fig, ax = plt.subplots(1, 3, figsize=(18, 6))

    plt.sca(ax[0])
    plt.imshow(rgb)
    plt.axis('off')
    plt.title('RGB')

    plt.sca(ax[1])
    plt.imshow(rgb)
    plt.imshow(mask_overlay, alpha=0.5)
    plt.axis('off')
    plt.title('vegetation segmentation')

    plt.sca(ax[2])
    plt.imshow(rgb)
    plt.imshow(inverse_mask_overlay, alpha=0.5)
    plt.axis('off')
    plt.title('non-veg segmentation')

    plt.show()

In [ ]:
def process_image(image_path):
    """
    Process a geospatial image to perform vegetation segmentation and visualization.

    Parameters:
    -----------
    image_path : str
        The file path to the geospatial image.

    Returns:
    --------
    rgb : numpy.ndarray
        The original RGB image.
    gsd : float
        The Ground Sampling Distance at the altitude of the image capture, in meters per pixel.
    area_of_tree : float
        The area of the tree in pixels.
    vegetation_mask : numpy.ndarray
        The binary mask indicating vegetation areas.
    mask_overlay : numpy.ndarray
        The RGB image with the vegetation mask overlay applied.
    inverse_mask_overlay : numpy.ndarray
        The RGB image with the non-vegetation mask overlay applied.
    """
    print(f"Processing image: {image_path}")

    # Get the bands of the image
    blue, green, red, rgb = get_bands(image_path)

    # Get the metadata of the image
    metadata = get_metadata(image_path)

    # Get the GSD of the image
    gsd, gsd_tree = get_gsd(metadata)

    # Get the kernel size for morphological operations
    area_of_tree, kernel_size = get_kernel_size(gsd_tree)

    # Calculate the VARI index
    vari = get_vari(blue, green, red)

    # Generate the vegetation and non-vegetation masks
    vegetation_mask, non_vegetation_mask = threshold(vari)

    # Apply smoothing and morphological operations to the vegetation mask
    smoothed_mask = smoothing(vegetation_mask)

    # Apply morphological operations to the smoothed mask
    filtered_mask = morphological_operations(smoothed_mask, kernel_size)

    # Apply the mask to the RGB image
    mask_overlay, inverse_mask_overlay = segmentation(filtered_mask, rgb)

    return rgb, gsd, area_of_tree, vegetation_mask, mask_overlay, inverse_mask_overlay

### Test out the image processing pipeline

Sanity check: Test the image processing pipeline with the image used in the rest of the notebooks.

In [ ]:
import os

image_path = "/content/DJI_0876.JPG" if os.path.exists(
    "/content/DJI_0876.JPG"
) else "data/presentation/8928dec4ddbffff/DJI_0876.JPG"

# Process the image
rgb, gsd, area_of_tree, vegetation_mask, mask_overlay, inverse_mask_overlay = process_image(
    image_path)

# Visualize the segmentation
visualize_segmentation(rgb, mask_overlay, inverse_mask_overlay)

## Image Processing

### Batch processing data

Consolidated a list of image file paths. 

The dataset contain 12 images randomly selected throughout the city of Vancouver, BC, Canada.

In [ ]:
import glob

# Define primary and secondary paths
primary_path = '/content/'  # Google Colab
secondary_path = 'data/presentation/*/*/'  # Repo

# Find images in both paths, Google Colab or relative in the repo
primary_images = glob.glob(os.path.join(primary_path, '**/*.JPG'),
                           recursive=True)
secondary_images = glob.glob(os.path.join(secondary_path, '*.JPG'))

# Combine both lists, ensuring no duplicates
images = list(set(primary_images + secondary_images))

Batch process all the data!

*This may take a while...*

In [ ]:
num_images = 0
total_trees = 0
gsd_sum = 0

for image_path in images:
    rgb, gsd, area_of_tree, vegetation_mask, mask_overlay, inverse_mask_overlay = process_image(
        image_path)

    # Count the number of pixels assocaited with vegetation
    num_vegetation_pixels = np.nansum(vegetation_mask)
    # Count the number of trees in the image
    n_trees = num_vegetation_pixels / (area_of_tree)
    print(f"Number of trees: {int(n_trees)}")

    visualize_segmentation(rgb, mask_overlay, inverse_mask_overlay)

    total_trees += n_trees
    num_images += 1
    gsd_sum += gsd

## Science time!
### Estimate carbon absorbed

Calculate the average number of trees per image and the average amount of carbon absorbed by these trees based on their CO2 absorption rate.

In [ ]:
# Calculate the number of trees in the image: number of vegetation pixels / area of tree
avg_n_trees = total_trees / num_images
print(f'Average number of trees per scene: {n_trees:.0f}')

# CO2 absorbed per tree in kg (https://ecotree.green/en/how-much-co2-does-a-tree-absorb#:~:text=A%20tree%20absorbs%20approximately%2025kg%20of%20CO2%20per%20year&text=But%20really%20a%20tree%20absorbs,a%20tree%20absorbs%20so%20interesting.)
carbon_absorbed_per_tree = 25

# Calculate the amount of carbon absorbed by the trees
carbon_absorbed = n_trees * carbon_absorbed_per_tree

print(f'Average carbon absorbed per scene: {carbon_absorbed:.0f} kg')

Now, let's roughly extrapolate this analysis for the entire city.

First, let's find the total area of the average image (i.e., the footprint of the camera).

In [ ]:
# Count total pixels in an image
total_pixels = rgb.size / 3

# The average GSD over all images
avg_gsd = gsd_sum / num_images
print(f'Average GSD: {round(gsd * 100, 2)} cm/px')

# Since the camera is pointed down we can assume the footprint is a rectangle
# Calculate the footprint of the camera
footprint_area = total_pixels * gsd**2

print(f'Footprint area: {footprint_area:.2f} m^2')

The city of Vancouver has an area of ~115.18km^2

In [ ]:
area = 115.18 * 1000 * 1000  # 115.18km^2

# Number of scenes it would take to cover the city = area of city / area of scene
num_scenes = area / footprint_area

print(f'Number of scenes to cover city: {num_scenes:.0f}')

Extrapolate the carbon absorbed by the city (Vancouver, BC, Canada)

In [ ]:
# Calculate the carbon absorbed by the city
carbon_absorbed_city = carbon_absorbed * num_scenes

print(
    f"Vancouver's trees absorb {carbon_absorbed_city / 1000:.0f} tonnes of CO2 per year"
)
print(f'Vancouver emits 28,000 tonnes of CO2 per year')
print(
    f"Vancouver's trees absorb ~{carbon_absorbed_city / 1000 / 28e3*100:.2f}% of the CO2 emitted by the city"
)

### Conclusion

The City of Vancouver emits 28,048 tonnes CO2e ([reference](https://metrovancouver.org/services/air-quality-climate-action/Documents/annual-corporate-energy-and-greenhouse-gas-emissions-management-report-2018-2022.pdf)). That means that these trees are likely absorbing a significant portion of Vancouver's greenhouse gases!

### Drawbacks
Here let's talk about and show where this method fails:
- Segmentation method heavily dependant on: colour of the tree, shadows, sensor calibration, time of day, lighting, weather, seasons, tree species, size of tree, shape of tree?
  - Adaptive thresholding helps with variable lighting conditions
  - Temporal analysis would help distinguishing between different types of vegetation
- Difficult to parse trees from grass and shrubs
  - Could use texture analysis to parse these out  
- Counting trees heavily dependant of tree size estimate
- Carbon capture only an estimate
- Working with small number statistics, extrapolation is uncertain